Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-25 16:37:41--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-25 16:37:42--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   188KB/s    in 1.1s    

2021-10-25 16:37:43 (188 KB/s) - ‘data.zip’ saved [217313/217313]



In [ ]:
!unzip -q data.zip

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 13.2 MB/s 
     |████████████████████████████████| 596 kB 41.0 MB/s 
     |████████████████████████████████| 895 kB 30.9 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 


In [ ]:
# Additional data

!gdown https://drive.google.com/uc?id=1TsXZPMFBY1ZAMCZrLFh4UN9Xb6uKmf9u

# URL: https://drive.google.com/file/d/1TsXZPMFBY1ZAMCZrLFh4UN9Xb6uKmf9u/view?usp=sharing

!unzip -q my_wic_mcl_data.zip


Downloading...
From: https://drive.google.com/uc?id=1TsXZPMFBY1ZAMCZrLFh4UN9Xb6uKmf9u
To: /content/my_wic_mcl_data.zip
100% 1.11M/1.11M [00:00<00:00, 74.4MB/s]


## Imports

In [ ]:
## Import relevant packages

import os
import pandas as pd
import numpy as np

import torch

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler

import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
from transformers import get_scheduler, AdamW


from argparse import ArgumentParser
from pdb import set_trace
from tqdm import tqdm
import time, datetime

import random
import json

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def seed_everything(seed=772):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
## Various utility functions

"""checking arguments"""
def check_args(args):
	# --result_dir
	check_folder(os.path.join(args.out_dir, args.models_dir))
	check_folder(args.tmp_dir)

	# --epoch
	try:
			assert args.epochs >= 1
	except:
			print('number of epochs must be larger than or equal to one')

	# --batch_size
	try:
			assert args.batch_size >= 1
	except:
			print('batch size must be larger than or equal to one')
	 
	# --data_path
	# assert os.path.isfile(args.data_path), "data_path not found"
	# assert os.path.isfile(args.gold_path), "gold_path not found"

  # --train_data_path
	assert os.path.isfile(args.train_data_path), "train_data_path not found"
	assert os.path.isfile(args.train_gold_path), "train_gold_path not found"

  # --val_data_path
	assert os.path.isfile(args.val_data_path), "val_data_path not found"
	assert os.path.isfile(args.val_gold_path), "val_gold_path not found"


	return args

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
												filename = "{}/{}.log".format(args.out_dir, phase),
												format = '%(asctime)s - %(message)s', 
												datefmt='%d-%b-%y %H:%M:%S')
	return logging.getLogger(phase)

## Args

In [ ]:
def parse_args():
	parser = ArgumentParser(description='Word Meaning Comparison')
	# parser.add_argument('--data_path', '-d', type=str, default='data/train/train.data.txt')
	# parser.add_argument('--gold_path', '-l', type=str, default='data/train/train.gold.txt')
	
	parser.add_argument('--train_data_path', '-td', type=str, default='data/train/train.data.txt')
	parser.add_argument('--train_gold_path', '-tl', type=str, default='data/train/train.gold.txt')
	
	parser.add_argument('--val_data_path', '-vd', type=str, default='data/validation/validation.data.txt')
	parser.add_argument('--val_gold_path', '-vl', type=str, default='data/validation/validation.gold.txt')
	
	parser.add_argument('--add_train_data_path', '-atd', type=str, default='my_wic_mcl_data/training.en-en.data')
	parser.add_argument('--add_train_gold_path', '-atl', type=str, default='my_wic_mcl_data/training.en-en.gold')
	parser.add_argument('--add_val_data_path', '-avd', type=str, default='my_wic_mcl_data/dev.en-en.data')
	parser.add_argument('--add_val_gold_path', '-avl', type=str, default='my_wic_mcl_data/dev.en-en.gold')
	

	parser.add_argument('--model', '-m', type=str, default='bert-base-cased', choices=['bert-base-uncased','bert-base-cased'])
	parser.add_argument('--vocab', '-v', type=str, default='bert-base-cased', choices=['bert-base-uncased','bert-base-cased'])

	parser.add_argument('--out_dir', type=str, default='out')
	parser.add_argument('--models_dir', type=str, default='models')
	parser.add_argument('--tmp_dir', type=str, default='tmp')
	
	parser.add_argument('--gpu', type=int, default=1)
	parser.add_argument('--batch_size', type=int, default=16)
	parser.add_argument('--epochs', type=int, default=3)
	parser.add_argument('--lr', type=float, default=2e-5)


	return check_args(parser.parse_known_args()[0])


args = parse_args()
print(args)

Namespace(add_train_data_path='my_wic_mcl_data/training.en-en.data', add_train_gold_path='my_wic_mcl_data/training.en-en.gold', add_val_data_path='my_wic_mcl_data/dev.en-en.data', add_val_gold_path='my_wic_mcl_data/dev.en-en.gold', batch_size=16, epochs=3, gpu=1, lr=2e-05, model='bert-base-cased', models_dir='models', out_dir='out', tmp_dir='tmp', train_data_path='data/train/train.data.txt', train_gold_path='data/train/train.gold.txt', val_data_path='data/validation/validation.data.txt', val_gold_path='data/validation/validation.gold.txt', vocab='bert-base-cased')


In [ ]:
device = torch.device('cuda' if args.gpu and torch.cuda.is_available() else 'cpu')

# Helper constants

In [ ]:
label_encode = {
    'F': 0,
    'T': 1
}

label_decode = {
    0: 'F',
    1: 'T'
}

# Data

In [ ]:
class BertDataset(Dataset):
  def __init__(self, vocab, data_path, gold_path=None, json_format=False):
    self.label_encode = {
                          'F': 0,
                          'T': 1
                        }
    if vocab in ["bert-base-uncased", "bert-base-cased"]:
      self.tokenizer = BertTokenizerFast.from_pretrained(vocab)
    else:
      raise RuntimeError(f"Invalid voacb. Got: {vocab}")

    if json_format:    # for WiC-MLM dataset
      with open(data_path, 'r') as f:
        data = json.load(f)

      self.sentences = [(s['sentence1'], s['sentence2']) for s in data]
      self.idx_ranges = [((int(s['start1']), int(s['end1'])), (int(s['start2']), int(s['end2']))) for s in data]
      
      self.labels=None
      if gold_path is not None:
        with open(gold_path, 'r') as f:
          data = json.load(f)
        self.labels = [self.label_encode[s['tag']] for s in data]
        
    else:
      df = pd.read_csv(data_path, sep='\t', names=['word', 'pos', 'idx', 's1', 's2'])

      self.sentences = [(s1, s2) for s1, s2 in zip(df["s1"], df["s2"])]
      #self.idx_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df["idx1_start"], df["idx1_end"], df["idx2_start"], df["idx2_end"])]
      
      self.labels=None
      if gold_path is not None:
        df = pd.read_csv(gold_path, names=['label'])
        df['label'] = df['label'].replace(self.label_encode)
        self.labels = df["label"].tolist()

    print(f"vocab: {vocab}, json: {json_format}")
    print(self.sentences[0],self.sentences[1],self.sentences[2])
    print(len(self.sentences))
                   
  def tokenize(self, s1, s2):
    return self.tokenizer(s1, s2,
                          add_special_tokens = True,
                          padding="max_length",
                          max_length=128,
                          #padding="longest",
                          truncation=True,
                          #return_offsets_mapping=True,
                          #return_tensors="pt"
                          )


  def __len__(self):
    return len(self.sentences)
  
  def __getitem__(self, index):
    s1, s2 = self.sentences[index]
    tokenized = self.tokenize(s1, s2)

    input_ids = torch.tensor(tokenized['input_ids'])
    token_type_ids = torch.tensor(tokenized['token_type_ids'])
    attention_mask = torch.tensor(tokenized['attention_mask'])

    label = self.labels[index] if self.labels is not None else -1

    return (input_ids, token_type_ids, attention_mask), torch.tensor(label, dtype=torch.int64).to(device)


In [ ]:
if True:
  dt = BertDataset(data_path=args.train_data_path, gold_path=args.train_gold_path, vocab=args.vocab)
  #print(dt.tokenizer.decode(dt.__getitem__(0)[0]['input_ids'][0]))
  print(dt.__getitem__(0)[0][0].size())
  print(dt.__getitem__(0)[-1])
  #del dt

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab: bert-base-cased, json: False
('You must carry your camping gear .', 'Sound carries well over water .') ('Messages must go through diplomatic channels .', 'Do you think the sofa will go through the door ?') ('Break an alibi .', 'The wholesaler broke the container loads into palettes and boxes for local retailers .')
5428
torch.Size([128])
tensor(0, device='cuda:0')


In [ ]:
seed_everything()

train_data = BertDataset(data_path=args.train_data_path,
                         gold_path=args.train_gold_path,
                         vocab=args.vocab)

train_dloader = DataLoader(train_data, 
                           batch_size=args.batch_size,
                           shuffle=True,
                           #pin_memory=True,
                           #num_workers=0,
                           )

#####

val_data = BertDataset(data_path=args.val_data_path,
                         gold_path=args.val_gold_path,
                         vocab=args.vocab)

val_dloader = DataLoader(val_data, 
                           batch_size=len(val_data), #args.batch_size,
                           shuffle=False,
                           #pin_memory=True,
                           #num_workers=0,
                           )

vocab: bert-base-cased, json: False
('You must carry your camping gear .', 'Sound carries well over water .') ('Messages must go through diplomatic channels .', 'Do you think the sofa will go through the door ?') ('Break an alibi .', 'The wholesaler broke the container loads into palettes and boxes for local retailers .')
5428
vocab: bert-base-cased, json: False
('Room and board .', 'He nailed boards across the windows .') ('Circulate a rumor .', 'This letter is being circulated among the faculty .') ('Hook a fish .', 'He hooked a snake accidentally , and was so scared he dropped his rod into the water .')
638


In [ ]:
seed_everything()

add_train_data = BertDataset(data_path=args.add_train_data_path,
                         gold_path=args.add_train_gold_path,
                         vocab=args.vocab,
                         json_format=True)

add_train_dloader = DataLoader(add_train_data, 
                           batch_size=args.batch_size,
                           shuffle=True,
                           #pin_memory=True,
                           #num_workers=0,
                           )

#####

add_val_data = BertDataset(data_path=args.add_val_data_path,
                         gold_path=args.add_val_gold_path,
                         vocab=args.vocab,
                         json_format=True)

add_val_dloader = DataLoader(add_val_data, 
                           batch_size=args.batch_size,
                           shuffle=True,
                           #pin_memory=True,
                           #num_workers=0,
                           )

vocab: bert-base-cased, json: True
('In that context of coordination and integration, Bolivia holds a key play in any process of infrastructure development.', 'A musical play on the same subject was also staged in Kathmandu for three days.') ('In that context of coordination and integration, Bolivia holds a key play in any process of infrastructure development.', 'In schools, when water is needed, it is girls who are sent to fetch it, taking time away from their studies and play.') ('We would also like to convey our esteem and congratulations to fraternal Lebanon and its people on the unconditional liberation of its southern part.', 'Father Lini said that, because of that, the United Nations has a very special place in the affections and esteem of the people of Vanuatu.')
8000
vocab: bert-base-cased, json: True
('No clause in a contract shall be interpreted as evading the responsibility of superiors under international law.', 'While fully aware that bishops and major superiors of relig

In [ ]:

for t in train_dloader:
  print(t[0][0].size())
  for el in t[0][0]:
    print(dt.tokenizer.decode(el))
  break


torch.Size([16, 128])
[CLS] The annual report has been my baby since September. [SEP] When is your baby due? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] We need to replace the secretary that left a month ago. [SEP] Mary replaced Susan as the team's captain and the highest - ranked player in the school. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

# Model

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    args.model, 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = False, 
    )

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Fine Tune

In [ ]:
# optimizer = O.AdamW(model.parameters(), lr=args.lr) #, weight_decay=args.lr_decay)
optimizer = AdamW(model.parameters(), lr=args.lr)


In [ ]:

model.to(device)
#train_dloader.to(device)
#val_dloader.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
'''
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=args.epochs*len(train_dloader)
    # num_training_steps=2*args.epochs*len(train_dloader)  ## hack!
)
'''

'\nlr_scheduler = get_scheduler(\n    "linear",\n    optimizer=optimizer,\n    num_warmup_steps=0,\n    num_training_steps=args.epochs*len(train_dloader)\n    # num_training_steps=2*args.epochs*len(train_dloader)  ## hack!\n)\n'

In [ ]:
def accuracy(y_prob, y_true):
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    # y_prob = y_prob > 0.5
    return (y_true == y_prob).sum().item() / y_true.size(0)

def train(model, it, optimizer):
    model.train()
    ep_loss = 0.0
    ep_acc = 0.0
    for b in tqdm(it):
        optimizer.zero_grad() #first??

        (input_ids, token_type_ids, attention_mask), labels = b
        input_ids, token_type_ids, attention_mask = input_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
        
        pred = model(input_ids=input_ids,
                      token_type_ids=token_type_ids,
                      attention_mask=attention_mask,
                      labels=labels
                      #labels=F.one_hot(labels, 2)
                      )
        loss = pred.loss
        acc = accuracy(torch.argmax(pred.logits, dim=1), labels)
        
        loss.backward()  
        #nn.utils.clip_grad_value_(model.parameters(), 10)
        #nn.utils.clip_grad_norm_(model.parameters(), 10)

        optimizer.step()
        lr_scheduler.step()

        ep_loss += loss.item()
        ep_acc += acc
        
        # get back to cpu
        input_ids, token_type_ids, attention_mask = input_ids.cpu(), token_type_ids.cpu(), attention_mask.cpu()

    return ep_loss/ len(it), ep_acc/ len(it)  # todo: /len(it) ??

def evaluate(model, it):
    model.eval()
    ep_loss = 0.0
    ep_acc = 0.0
    with torch.no_grad():
	    for b in it:
	        (input_ids, token_type_ids, attention_mask), labels = b
	        input_ids, token_type_ids, attention_mask = input_ids.to(device), token_type_ids.to(device), attention_mask.to(device)

	        pred = model(input_ids=input_ids,
                      token_type_ids=token_type_ids,
                      attention_mask=attention_mask,
                      )
	        
	        #loss = pred.loss
	        #ep_loss += loss.item()
	        
	        acc = accuracy(torch.argmax(pred.logits, dim=1), labels)
	        ep_acc += acc
	        input_ids, token_type_ids, attention_mask = input_ids.cpu(), token_type_ids.cpu(), attention_mask.cpu()

    return ep_loss/ len(it), ep_acc/ len(it)

In [ ]:
'''
seed_everything()
for ep in range(args.epochs):
    print ('training epoch {}... [lr={}]'.format(ep, optimizer.param_groups[0]['lr']))
    tr_loss, tr_acc = train(model, train_dloader, optimizer)
    print ('TRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))
    
    vl_loss, vl_acc = evaluate(model, val_dloader)
    print('VALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))

'''

"\nseed_everything()\nfor ep in range(args.epochs):\n    print ('training epoch {}... [lr={}]'.format(ep, optimizer.param_groups[0]['lr']))\n    tr_loss, tr_acc = train(model, train_dloader, optimizer)\n    print ('TRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))\n    \n    vl_loss, vl_acc = evaluate(model, val_dloader)\n    print('VALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))\n\n"

### New
Note: `loss` for VAL set is not computed

In [ ]:
optimizer = AdamW(model.parameters(), lr=0.49*args.lr)  # reduce lr -- shouldn't affect much!!

seed_everything()

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    #num_training_steps=args.epochs*(len(train_dloader)+len(add_train_dloader)+len(add_val_dloader))
    num_training_steps=2*1*(len(add_train_dloader)+len(add_val_dloader))  ##hack !!
)

for ep in range(1):
    print ('training epoch {}... [lr={}]'.format(ep, optimizer.param_groups[0]['lr']))

    tr_loss, tr_acc = train(model, add_train_dloader, optimizer)
    print ('TRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))
    vl_loss, vl_acc = evaluate(model, val_dloader)
    print('VALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))

    tr_loss, tr_acc = train(model, add_val_dloader, optimizer)
    print ('TRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))
    vl_loss, vl_acc = evaluate(model, val_dloader)
    print('VALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))



training epoch 0... [lr=9.800000000000001e-06]


100%|██████████| 500/500 [05:54<00:00,  1.41it/s]


TRAIN: loss 0.5240 acc 0.7431
VALID: loss 0.0000 acc 0.6771


100%|██████████| 63/63 [00:44<00:00,  1.42it/s]


TRAIN: loss 0.4087 acc 0.8333
VALID: loss 0.0000 acc 0.6755


In [ ]:
optimizer = AdamW(model.parameters(), lr=args.lr)

seed_everything()

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=2*args.epochs*(len(train_dloader))
)
for ep in range(args.epochs):
    print ('training epoch {}... [lr={}]'.format(ep, optimizer.param_groups[0]['lr']))

    tr_loss, tr_acc = train(model, train_dloader, optimizer)
    print ('TRAIN: loss %.4f acc %.4f' % (tr_loss, tr_acc))
    vl_loss, vl_acc = evaluate(model, val_dloader)
    print('VALID: loss %.4f acc %.4f' % (vl_loss, vl_acc))



training epoch 0... [lr=2e-05]


100%|██████████| 340/340 [03:57<00:00,  1.43it/s]


TRAIN: loss 0.5420 acc 0.7346
VALID: loss 0.0000 acc 0.7226
training epoch 1... [lr=1.6666666666666667e-05]


100%|██████████| 340/340 [03:57<00:00,  1.43it/s]


TRAIN: loss 0.2820 acc 0.8910
VALID: loss 0.0000 acc 0.7257
training epoch 2... [lr=1.3333333333333333e-05]


100%|██████████| 340/340 [03:57<00:00,  1.43it/s]


TRAIN: loss 0.1133 acc 0.9675
VALID: loss 0.0000 acc 0.7210


# `del` model and data to free up memory
danger!

In [ ]:
'''
torch.cuda.empty_cache()
'''

'\ntorch.cuda.empty_cache()\n'

In [ ]:
"""
del model
del train_data
del val_data
"""

'\ndel model\ndel train_data\ndel val_data\n'

# Store
Errors in this section, if any, can be ignored. In fact, model training is complete before this section.

In [ ]:
if True:
  save_model = {
      'args': args,
      'model': model,
      'model_state': model.state_dict()
  }

  torch.save(save_model,
            os.path.join(args.out_dir, args.models_dir, "model-{}.pt".format(time.time()))
            )

In [ ]:
if False:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DRIVE_PREFIX="/content/drive/MyDrive/col772_a3.2"

!cd out && zip -r ../2018MT10770_B_model.zip models/ && cd ../
!echo "copying..."
!cp "/content/2018MT10770_B_model.zip" $DRIVE_PREFIX"/models/"


  adding: models/ (stored 0%)
  adding: models/model-1635181112.7060943.pt (deflated 7%)
copying...


# Expermiments :) 
There is nothing useful in this section.
Please leave it collapsed.

In [ ]:
'''
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')



"""
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)
"""

# https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209

"""
encoding = tokenizer.encode_plus(
                                 text,
                                 add_special_tokens = True,
                                 truncation = True, 
                                 padding = "max_length", 
                                 return_attention_mask = True, 
                                 return_tensors = "pt"
                                 )
"""

# https://huggingface.co/transformers/preprocessing.html

batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                    "And the very very last one",
                    "The Obamagate."
                   ]

inputs = tokenizer(
                   batch_sentences,
                   add_special_tokens = True,
                   padding='max_length',
                   truncation=True,
                   return_tensors="pt"
                   )

print(inputs)

for s in inputs["input_ids"]:
  print(tokenizer.decode(s))
'''

'\nfrom transformers import BertTokenizer, BertModel\nimport torch\n\ntokenizer = BertTokenizer.from_pretrained(\'bert-base-uncased\')\nmodel = BertModel.from_pretrained(\'bert-base-uncased\')\n\n\n\n"""\ninputs = tokenizer("Hello, my dog is cute", return_tensors="pt")\noutputs = model(**inputs)\n"""\n\n# https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209\n\n"""\nencoding = tokenizer.encode_plus(\n                                 text,\n                                 add_special_tokens = True,\n                                 truncation = True, \n                                 padding = "max_length", \n                                 return_attention_mask = True, \n                                 return_tensors = "pt"\n                                 )\n"""\n\n# https://huggingface.co/transformers/preprocessing.html\n\nbatch_sentences = ["Hello I\'m a single sentence",\n                    "And another sentence",\n             

In [ ]:
'''
from transformers import BertTokenizerFast, BertModel
import torch

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')



# https://huggingface.co/transformers/preprocessing.html

batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                   ]
batch_sentences_2 = ["And the very very last one",
                    "The Obamagate."]

inputs = tokenizer(
                   batch_sentences, batch_sentences_2,
                   add_special_tokens = True,
                   padding='max_length',
                   truncation=True,
                   return_offsets_mapping=True,
                   return_tensors="pt"
                   )

print(inputs["offset_mapping"][0][:100])

print(inputs['input_ids'].size())

for s in inputs["input_ids"]:
  print(tokenizer.decode(s))
'''

'\nfrom transformers import BertTokenizerFast, BertModel\nimport torch\n\ntokenizer = BertTokenizerFast.from_pretrained(\'bert-base-uncased\')\nmodel = BertModel.from_pretrained(\'bert-base-uncased\')\n\n\n\n# https://huggingface.co/transformers/preprocessing.html\n\nbatch_sentences = ["Hello I\'m a single sentence",\n                    "And another sentence",\n                   ]\nbatch_sentences_2 = ["And the very very last one",\n                    "The Obamagate."]\n\ninputs = tokenizer(\n                   batch_sentences, batch_sentences_2,\n                   add_special_tokens = True,\n                   padding=\'max_length\',\n                   truncation=True,\n                   return_offsets_mapping=True,\n                   return_tensors="pt"\n                   )\n\nprint(inputs["offset_mapping"][0][:100])\n\nprint(inputs[\'input_ids\'].size())\n\nfor s in inputs["input_ids"]:\n  print(tokenizer.decode(s))\n'

In [ ]:
'''
outputs = model(**inputs)
'''

'\noutputs = model(**inputs)\n'

In [ ]:
'''outputs'''

'outputs'

In [ ]:
'''
#https://github.com/huggingface/transformers/issues/3640

import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModel, AutoTokenizer, AutoModelWithLMHead, ElectraModel, ElectraForMaskedLM

MODEL_PATH = 'bert-base-uncased'

VOCAB = MODEL_PATH

print('== tokenizing ===')
tokenizer = BertTokenizer.from_pretrained(VOCAB)

# Tokenized input
text = "[MASK] the table."
## set	V	0-1	Set the table .	To set glass in a sash .


inputs = tokenizer.encode_plus(text, return_tensors="pt")

masked_index = 1

model = BertForMaskedLM.from_pretrained(MODEL_PATH)
model.eval()

print('== LM predicting ===')
# Predict all tokens
predictions = model(**inputs)[0]

print(predictions.size())

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
print(predicted_index)
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
print('predicted_token', predicted_token)
'''

'\n#https://github.com/huggingface/transformers/issues/3640\n\nimport torch\nfrom transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModel, AutoTokenizer, AutoModelWithLMHead, ElectraModel, ElectraForMaskedLM\n\nMODEL_PATH = \'bert-base-uncased\'\n\nVOCAB = MODEL_PATH\n\nprint(\'== tokenizing ===\')\ntokenizer = BertTokenizer.from_pretrained(VOCAB)\n\n# Tokenized input\ntext = "[MASK] the table."\n## set\tV\t0-1\tSet the table .\tTo set glass in a sash .\n\n\ninputs = tokenizer.encode_plus(text, return_tensors="pt")\n\nmasked_index = 1\n\nmodel = BertForMaskedLM.from_pretrained(MODEL_PATH)\nmodel.eval()\n\nprint(\'== LM predicting ===\')\n# Predict all tokens\npredictions = model(**inputs)[0]\n\nprint(predictions.size())\n\n# confirm we were able to predict \'henson\'\npredicted_index = torch.argmax(predictions[0, masked_index]).item()\nprint(predicted_index)\npredicted_token = tokenizer.convert_ids_to_tokens([predicted_index])\nprint(\'predicted_token\', predicted_toke

In [ ]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)



In [ ]:
'''
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r USERID_A_model.zip **

## Upload it to Google drive and ensure that the testing notebook uses the correct link
'''

'\n## Zip the final model and all the required files, such as vocabulary\n# Replace USERID with your own, such as 2017CSZ8058\n!zip -r USERID_A_model.zip **\n\n## Upload it to Google drive and ensure that the testing notebook uses the correct link\n'